In [ ]:
!nvidia-smi

Mon May 24 08:28:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip3 install -U gitpython
!pip3 install transformers
!pip3 install urllib3

     |████████████████████████████████| 174kB 3.0MB/s 
     |████████████████████████████████| 71kB 8.1MB/s 
     |████████████████████████████████| 2.3MB 3.0MB/s 
     |████████████████████████████████| 901kB 18.5MB/s 
     |████████████████████████████████| 3.3MB 19.9MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd 
from tqdm import tqdm
from tqdm import trange
import gc
import copy
import os
import re
import string
import random
import math
import time
import datetime

from collections import Counter
import itertools

import torch
from transformers import AutoModelWithLMHead, AutoTokenizer, GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, random_split, DataLoader, RandomSampler, SequentialSampler
from torch.nn import functional as F

pd.set_option('display.max_rows', 50)

In [ ]:
device = torch.device('cuda')

In [ ]:
data_path = "./drive/MyDrive/anna/data/poems.csv"

In [ ]:
data = pd.read_csv(data_path)

In [ ]:
data.fillna("", inplace=True)

In [ ]:
def preprocess(df):
    texts = df['text'].values
    for i in range(len(texts)):
        texts[i] = texts[i].lower()
        texts[i] = re.sub("[0-9]", "", texts[i])
        texts[i] = re.sub(' +', ' ', texts[i])
    authors = df['author'].values
    for i in range(len(authors)):
        authors[i] = authors[i].lower()
        authors[i] = re.sub("[0-9]", "", authors[i])
        authors[i] = re.sub('\s', ' ', authors[i])
        
    df['text'] = texts

preprocess(data)
data

,author,text
0,марина копсергенова,\r\nдетки мои живите неспеша чтоб пела радост...
1,владислав иванов,каждый по своему проводит свою ночь\r\nкто спи...
2,лия эль,нежно так я словами касался предплечий\r\nих н...
3,елена кедровая шишка,ты как всегда меня поймешь\r\nи как всегда про...
4,леонид хегай,укусить бы кого нибудь больно\r\nиль повыть пр...
...,...,...
164438,кирилл кузнечихин,пародия на дворовые романсы\r\nваши ножки чуд...
164439,владислав никоненко,не надо нагло нарушать\r\nзаконы мойте руки\r\...
164440,наталья потапенко,магда нравится так что же здесь странного\r\nт...
164441,фатима дадаева,нам пустоту рождает пустота\r\nи голова нужна ...


In [ ]:
model = AutoModelWithLMHead.from_pretrained('sberbank-ai/rugpt3large_based_on_gpt2')

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:810: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
model = model.to(device)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/rugpt3large_based_on_gpt2')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### **ОБУЧЕНИЕ**

In [ ]:
#data.drop(data.head(84443).index, inplace=True)

In [ ]:
raw = data['text'].to_list()

In [ ]:
authors = data['author'].to_list()

In [ ]:
tokens = [tokenizer.encode(text) for text in raw]

In [ ]:
#validation_start_index = int(len(raw) * 0.1)
#validation_start_index

In [ ]:
train_lengths = [len(token) for token in tqdm(tokens)]

100%|██████████| 164443/164443 [00:00<00:00, 2721867.45it/s]


In [ ]:
print(tokenizer.vocab_size)

50257


In [ ]:
batch_size = 1
random_seed = 420
torch.manual_seed(random_seed)
max_len = 50

In [ ]:
special_tokens_dict = {'pad_token': '<PAD>', 'bos_token': '<BOS>', 'eos_token': '<EOS>'}

In [ ]:
pad_index = 50258
boc_index = 50259
eoc_index = 50260

In [ ]:
num_added_tokens = tokenizer.add_special_tokens(special_tokens_dict)

In [ ]:
class PoetryDataset(Dataset):
    def __init__(self, texts, authors, tokenizer, max_length):
        self.tokenizer = tokenizer
        self.texts = texts
        self.authors = authors
        self.max_len = max_length

    def __len__(self):        
        return len(self.texts)
    
    def __getitem__(self, index):

        text = self.texts[index]
        author = self.authors[index]
        text = f'<BOC>{author}<EOC>{text}'
        
        enc_dict = tokenizer(text, truncation=True, max_length=self.max_len)
        tokenized = enc_dict['input_ids']
        
        return torch.tensor(tokenized).long()

In [ ]:
poems_train = PoetryDataset(raw[100000:], authors, tokenizer, max_len)
poems_valid = PoetryDataset(raw[::10], authors[::10], tokenizer, max_len)

len(poems_train), len(poems_valid)

(64443, 16445)

In [ ]:
train_loader = DataLoader(poems_train,
                          batch_size=batch_size, 
                          shuffle=True)

validation_loader = DataLoader(poems_valid, 
                               batch_size=batch_size, 
                               shuffle=True)

In [ ]:
num_epochs = 1

In [ ]:
learning_rate = 0.0001
warmup_steps = 50
total_steps = len(train_loader) * num_epochs

In [ ]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50261, 1536)

In [ ]:
optimizer = AdamW(model.parameters(), lr=learning_rate)

In [ ]:
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=warmup_steps,
                                            num_training_steps=total_steps)

In [ ]:
def train(model, loader, optimizer, scheduler, last_n_losses=200, verbose=True):

    losses = []

    progress_bar = tqdm(total=len(loader.dataset), disable=not verbose, desc='Train')

    model.train()

    for tokens in loader:

        tokens = tokens.to(device)
        
        outputs = model(tokens)

        shift_tokens = tokens[..., 0:].contiguous()
        shift_logits = outputs.logits[..., :, :].contiguous()

        losses_vec = F.cross_entropy(shift_logits.view(-1, shift_logits.size(-1)), shift_tokens.view(-1), reduction="none")

        coeffs = torch.ones(list(losses_vec.size())[0]).cuda()
        indexes = (shift_tokens == 206).nonzero(as_tuple=True)[1]
        for index in indexes:
            if index > 1:
                coeffs[index - 1] *= 10
                coeffs[index - 2] *= 10

        losses_vec = coeffs*losses_vec

        loss = losses_vec.mean()
        
        losses.append(loss.item())

        loss.backward()
        optimizer.zero_grad()
        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix(loss=np.mean(losses[-last_n_losses:]),
                                 perplexity=np.exp(np.mean(losses[-last_n_losses:])))
        
        progress_bar.update()

    progress_bar.close()
    
    return losses

In [ ]:
def evaluate(model, loader, last_n_losses=200, verbose=True):

    losses = []

    progress_bar = tqdm(total=len(loader), disable=not verbose, desc='Evaluate')

    model.eval()

    for tokens in loader:

        tokens = tokens.to(device)
        
        outputs = model(tokens)

        shift_tokens = tokens[..., 0:].contiguous()
        shift_logits = outputs.logits[..., :, :].contiguous()

        losses_vec = F.cross_entropy(shift_logits.view(-1, shift_logits.size(-1)), shift_tokens.view(-1), reduction="none")

        coeffs = torch.ones(list(losses_vec.size())[0]).cuda()
        indexes = (shift_tokens == 206).nonzero(as_tuple=True)[1]
        for index in indexes:
            if index > 1:
                coeffs[index - 1] *= 10
                coeffs[index - 2] *= 10

        losses_vec = coeffs*losses_vec

        loss = losses_vec.mean()
        
        losses.append(loss.item())

        progress_bar.set_postfix(loss=np.mean(losses[-last_n_losses:]),
                                 perplexity=np.exp(np.mean(losses[-last_n_losses:])))
        
        progress_bar.update()

    progress_bar.close()
    
    return losses

In [ ]:
save_best_model_path = './drive/MyDrive/anna/best_model_state_dict.pth'
save_best_optimizer_path = './drive/MyDrive/anna/best_optimizer_state_dict.pth'
save_last_model_path = './drive/MyDrive/anna/last_model_state_dict.pth'
save_last_optimizer_path = './drive/MyDrive/anna/last_optimizer_state_dict.pth'

In [ ]:
# ЧИСТИМ ПАМЯТЬ ПЕРЕД ОБУЧЕНИЕМ

gc.collect()
torch.cuda.empty_cache()
model = model.to(device)

In [ ]:
train_losses = []
validation_losses = []

train_perplexities = []
validation_perplexities = []

best_validation_loss = 1e+6

for n_epoch in range(1, num_epochs + 1):
    
    epoch_train_losses = train(model, train_loader, optimizer, scheduler)

    torch.save(model.state_dict(), save_last_model_path)
    torch.save(optimizer.state_dict(), save_last_optimizer_path)

    epoch_validation_losses = evaluate(model, validation_loader)
    
    mean_train_loss = np.mean(epoch_train_losses)
    mean_validation_loss = np.mean(epoch_validation_losses)
    
    train_losses.extend(epoch_train_losses)
    train_perplexities.append(np.exp(mean_train_loss))
    
    validation_losses.extend(epoch_validation_losses)
    validation_perplexities.append(np.exp(mean_validation_loss))
    
    message = f'Epoch: {n_epoch}\n'
    message += f'Train: loss - {mean_train_loss:.4f} | perplexity - {train_perplexities[-1]:.3f}\n'
    message += f'Validation: loss - {mean_validation_loss:.4f} | perplexity - {validation_perplexities[-1]:.3f}'
    
    print(message)
    
    #if mean_validation_loss < best_validation_loss:
    #    
    #    best_validation_loss = mean_validation_loss
    #    
    #    torch.save(model.state_dict(), save_best_model_path)
    #    torch.save(optimizer.state_dict(), save_best_optimizer_path)
    #    
    #else:
    #    break
    

Evaluate: 100%|██████████| 16445/16445 [08:20<00:00, 32.84it/s, loss=31.1, perplexity=3.25e+13]


Epoch: 1
Train: loss - 29.8000 | perplexity - 8749519456776.827
Validation: loss - 30.9620 | perplexity - 27965489288727.348


In [ ]:
#torch.save(model.state_dict(), save_best_model_path)
#torch.save(optimizer.state_dict(), save_best_optimizer_path)

torch.save(model.state_dict(), save_last_model_path)
torch.save(optimizer.state_dict(), save_last_optimizer_path)

In [ ]:
model.load_state_dict(torch.load('./drive/MyDrive/anna/last_model_state_dict.pth'))

<All keys matched successfully>

In [ ]:
optimizer.load_state_dict(torch.load('./drive/MyDrive/anna/best_optimizer_state_dict.pth'))

In [ ]:
def write_poem(author, snippet, length, tokenizer, model):
    input_text = f'<BOC>{author}<EOC>{snippet}'
    inds = tokenizer(input_text, return_tensors='pt')['input_ids'].to(device)
    text = model.generate(
        input_ids=inds,
        max_length=length + len(inds),
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.9,
        no_repeat_ngram_size=20,
        pad_token_id = 50258
        )
    poem = tokenizer.decode(text[0])
    poem = poem.strip(f'<BOC>{author}<EOC>')
    return poem

In [ ]:
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50261, 1536)
    (wpe): Embedding(2048, 1536)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
     

In [ ]:
poem = write_poem('денис симагин', 'белеет парус одинокий\r\n', 50, tokenizer, model)
print(poem)

белеет парус одинокий
и ветер воет за окном
и в синем небе ярок и высок
и думаю что это я сон
и вижу наяву


In [ ]:
poem0 = write_poem('диана сусла', 'ночь улица фонарь аптека\r\n', 50, tokenizer, model)
print(poem0)

очь улица фонарь аптека
ты спишь и взгляд такой печальный
и вот опять от скуки и печали
я по тебе с тобой была
а я тебе


In [ ]:
poem1 = write_poem('дмитрий проклов', 'ночь улица фонарь аптека\r\n', 50, tokenizer, model)
print(poem1)

ночь улица фонарь аптека
я в ней опять один и пьян
а нет тебя мне скука
сижу опять один как перст
а в доме снова неза


In [ ]:
poem2 = write_poem('дмитрий проклов', 'с днём рождения поздравляю\r\n', 50, tokenizer, model)
print(poem2)

с днём рождения поздравляю
и хочу тебя поздравить
за тебя всегда молюсь
и за счастье быть с тобой хочу
я хочу тебя поцеловать
и счастливой сделать день


In [ ]:
text7 = f'<BOC>денис симагин<EOC>россия великая наша держава\r\n'

In [ ]:
input_ids7 = tokenizer(text7, return_tensors='pt')['input_ids'].to(device)
input_ids7

tensor([[   32,    38, 39463,    34,   280, 48071,  2964,  1225,   375,    32,
         20291,    39,    34,  1584,   274, 28652,  5571,  4985,   342,   206,
           203]], device='cuda:0')

In [ ]:
out7 = model.generate(
    input_ids=input_ids7,
    max_length=max_len + len(input_ids7),
    do_sample=True,
    no_repeat_ngram_size=20,
    pad_token_id = 50258
    )

In [ ]:
out7

tensor([[   32,    38, 39463,    34,   280, 48071,  2964,  1225,   375,    32,
         20291,    39,    34,  1584,   274, 28652,  5571,  4985,   342,   206,
           203,   265,  4410,   935,   309, 10020,   309, 10020,   206,   203,
           464,   704,   417,  6540,  2218, 12945,  1181,  3209,   206,   203,
          1024,   274,   694, 25278,  6917,  1244, 13713,   206,   203,   274,
          1315]], device='cuda:0')

In [ ]:
generated_text7 = tokenizer.decode(out7[0])
generated_text7

'<BOC>денис симагин<EOC>россия великая наша держава\r\nи пусть даже на краю на краю\r\nтебя я люблю мой родной человечек\r\nмоя ты любимая самая родимая\r\nя тебя'

In [ ]:
print(generated_text7)

<BOC>денис симагин<EOC>россия великая наша держава
и пусть даже на краю на краю
тебя я люблю мой родной человечек
моя ты любимая самая родимая
я тебя
